In [21]:
# power_rank.py

# NFL power ranking automation for Washington Post
# Jack Barry

# import stuff
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

############################### METHOD ###############################
# The inputs are easy:
 
# Team win percentage (W)
# Team Pythagorean win percentage (P)
# Opponent win percentage (O)
 
# Then it is just (W+P)/2 * (O/.500) * 100

# P = [(Points Scored)^2.37]/[(Points Scored)^2.37 + (Points Allowed)^2.37]

############################### FUNCTIONS ###############################

# function to read the regular season data for each year
def read_standings(url):
    # read data for given year
    dfs = pd.read_html(url, skiprows=0, flavor='bs4')
    # add separate dataframes (for separate conferences) to one DF
    year_df = pd.DataFrame()
    for df in dfs:
        year_df = year_df.append(df, ignore_index=True)
    # add column for year to df, fill with given year
    year_df = year_df[pd.isnull(year_df.W) == False]
    # fill null values
    year_df
    return year_df

# CALCULATE AVG WIN FOR EACH TEAM
def proj_lookup(team):
    gp = int(team.W+team.L)
    w = str(int(team.W))
    return proj_wins.loc[gp, w]

def opp_avg(team):
    opp = team.Opponent
    opp_pct = pwr.avg_wins[opp]
    return opp_pct

############################### IMPORT DATA ###############################

# read in schedule
sched = pd.read_csv('sched.csv')
sched.rename(columns={'Unnamed: 2':'Date','Unnamed: 5':'at'}, inplace=True)

####### UPDATE THIS LINE FOR WEEKLY SCHEDULE #######
week_sched = sched.loc[sched['Week'] == 17]
####### UPDATE THIS LINE FOR WEEKLY SCHEDULE #######

# create full list of matchups, with both home and away (for merging with standings)
matchups = pd.DataFrame()
matchups['Tm'] = pd.concat(objs=[week_sched.HomeTm, week_sched.VisTm], ignore_index=True)
matchups['Opp'] = pd.concat(objs=[week_sched.VisTm, week_sched.HomeTm], ignore_index=True)

# read in 2018 team data
stand_url = 'https://www.pro-football-reference.com/years/2018/'
stand = read_standings(stand_url)

stand.replace(to_replace={'New England Patriots*' : 'New England Patriots', 'Baltimore Ravens*' : 'Baltimore Ravens', 'Houston Texans*' : 'Houston Texans', 'Kansas City Chiefs*' : 'Kansas City Chiefs', 'Dallas Cowboys*' : 'Dallas Cowboys', 'Chicago Bears*' : 'Chicago Bears', 'New Orleans Saints*' : 'New Orleans Saints', 'Los Angeles Rams*' : 'Los Angeles Rams', 'Los Angeles Chargers+' : 'Los Angeles Chargers', 'Philadelphia Eagles+' : 'Philadelphia Eagles', 'Seattle Seahawks+' : 'Seattle Seahawks', 'Indianapolis Colts+' : 'Indianapolis Colts'}, inplace=True)

# STATIC CSVS
# read in csv for projected wins
proj_wins = pd.read_csv('win_pct.csv', encoding="ISO-8859-1")
proj_wins.set_index('GP', inplace=True)

# read in csv for all divisions
divs = pd.read_csv('divs.csv', encoding="ISO-8859-1")
divs.set_index('Tm', inplace=True)

# read in preseason projections
preseason = pd.read_csv('preseason.csv')
preseason.set_index('Team', inplace=True)

# MERGE INTO DATAFRAME
# merge schedule/matchup data
df = pd.merge(stand, matchups, on='Tm', how='outer')

df.set_index(keys='Tm', inplace=True)

# add preseason to df
df['preseason_proj_pct'] = preseason.pct

# add divisions for each team
df['division'] = divs['division']
df['conference'] = divs['conference']

# df.head(32)

In [22]:
############################### Update late MNF result ###############################
# # week 3: Pgh 30, Tampa 27

# df.loc['New Orleans Saints', 'W'] = 12
# df.loc['New Orleans Saints', 'PF'] = 459
# df.loc['New Orleans Saints', 'PA'] = 292

# df.loc['Carolina Panthers', 'L'] = 8
# df.loc['Carolina Panthers', 'PF'] = 333
# df.loc['Carolina Panthers', 'PA'] = 344
# df.head(32)

In [23]:
############################### CALCULATIONS ###############################
# calculate pythagorean win percentage for each team
df['Pyth_pct'] = df.PF**3/(df.PF**3 + df.PA**3)

# # look up historical win percentage based on record (not needed for playoffs)
# df['historical_win_pct'] = df.apply(proj_lookup, axis=1)

# MODEL PERCENTAGE: [avg(pyth %, historical %), preseason %]/2
df['model_pct'] = ((df['W-L%'] + df.Pyth_pct)/2)
# df['model_pct_old'] = (((df.historical_win_pct + df.Pyth_pct)/2)+df.preseason_proj_pct)/2

# reorder everything
# df = df[['conference', 'division', 'W', 'L', 'T', 'W-L%', 'PF', 'PA', 'PD', 'MoV', 'SoS', 'SRS', 'OSRS', 'DSRS', 'Opp', 'preseason_proj_pct', 'historical_win_pct', 'Pyth_pct', 'model_pct', 'model_pct_old']]

# ############################### CALCULATE RANKS ###############################

# # calculate (W+P)/2
# df['W_pct'] = (df['W-L%'] + df.Pyth)/2

# # Then it is just (W+P)/2 * (O/.500) * 100
# pwr['current_rank'] = pwr.avg_wins*(pwr.opp_avg_pct/.5)*100

# # average pythagorean wins and historical projections
# pwr['avg_wins'] = (pwr.Pyth + pwr.proj_wins)/2

# # find avg for each team/opponent
# opps['opp_rate'] = opps.apply(opp_avg, axis=1)

# # numbered ranks
# pwr['current_order'] = pwr.current_rank.rank(ascending=False)

# df.head()

In [24]:
df.loc['Baltimore Ravens', 'model_pct'] = 0.00001
df.loc['Houston Texans', 'model_pct'] = 0.00001
df.loc['Chicago Bears', 'model_pct'] = 0.00001
df.loc['Seattle Seahawks', 'model_pct'] = 0.00001
df.loc['Los Angeles Chargers', 'model_pct'] = 0.00001
df.loc['Dallas Cowboys', 'model_pct'] = 0.00001
df.loc['Philadelphia Eagles', 'model_pct'] = 0.00001
df.loc['Indianapolis Colts', 'model_pct'] = 0.00001

# print(len(df))
# print(df.index)
# df.head(42)

In [25]:
############################## SAVE DATA ###############################
date = datetime.date.isoformat(datetime.date.today())

# dataframes we need to keep
data = [df]

# create pickle object
with open('pkls/pwr_' +  date + '.pkl', 'wb') as handle:
    pickle.dump(data, handle)

In [66]:
################################### UNUSED CODE ####################################

########### AVERAGE ALL OPPONENTS WIN PERCENTS ###########
# # group each team's opponents together, calculate mean opp win%
# grouped = opps.groupby(by='Team')
# # add to pwr dataframe
# pwr['opp_avg_pct'] = grouped.opp_rate.mean()


########### READ SCHEDULE FROM PRO FOOTBALL REF ###########
# # function to read the regular season data for each year
# # not used
# def read_schedule(url, week):
#     # read data for given year
#     df = pd.read_html(url, skiprows=0, flavor='bs4')[0]
#     cols = ['Week', 'Day', 'Date', 'Time', 'Visitor', 'at', 'Home', 'preview', 'PtsW', 'PtsL', 'YdsW', 'TOW', 'YdsL', 'TOL']
#     df.columns = cols
#     df = df[df.preview =='preview']
#     df = df[df.Week == week]
#     # fill null values
#     return df